# Multiple resonators on a chip, with possibility of having holes in the resonator core

Ruirements: gdspy library.
The design is generated in the following order:


   *   **Ground and contact pads**. THe idea is that you should only specify the coordinates of the contact pads; whether they should be rotated should be decided by the code itself (if the contact pad is further that 900 $\mu m$ away, it's rotated). This is done by the functions 
   
   *Generate_Pads (sample_size_along_x_axis, sample_size_along_y_axis, [d], [d1], [d2], [layer], [contact pad coordinates])*. 
   
   
   *   **TL**. Connects two contact pads from left to right. Generally, in this code everything that involves coplanar waveguides should be drawn from left to right.
   
   *Generate_TL (contact_pad_1, contact_pad2, d, d1, d2)*
   
    
   *   **Resonators** The position is specified by the left lower corner in (x,y) format. 
   
       *Generate_Resonators (initial_point_array, [circular_claw_radius], [gap], [resonator_frequency], [mode (up or down)], [layer])*.
   
   *   **DC line**. At this point the we add resonators and dc bias lines to the sample cell.
   
   *Generate_Loops* ([coordinates of the second end; the first end automatically gets connected to a contact pad], [modes], [d], [d1], [d2], [d for the contact pad], [d1 for the contact pad], [d2 for the contact pad], [array of arc coordinates], [array of arc angles], layer)
   
   
   *   **Ground grid**. Can be generated only after everything else is in place
   
   *Generate_Grid* (sample_size_along_x, sample_size_along_y, layer1, layer2)
   
 IF you want to change anything, unfortunately you'll have to do a restart.

In [1]:
!pip install gdspy

In [2]:
from design_res3 import *
from resonators3 import*
import gdspy
import random as rd

In [3]:
class Sample:
    
    #Содержит функции, генерирующие все площадки, TL, все джозефсоны, все петельки
    
    def __init__(self, number):
        self.name = 'sample' + str(number)
        self.cell = gdspy.Cell(self.name)
    
    def Generate_Pads(self, a, b, number, d_arr, d1_arr, d2_arr, layer, coords = []):
        self.Pads = Pads(a, b, number, d_arr, d1_arr, d2_arr, layer, coords)
        self.Pads_cell = gdspy.Cell('Pads')
        self.Pads_cell.add(self.Pads.Generate_Ground())
        self.cell.add(gdspy.CellReference(self.Pads_cell, (0, 0)))
        
    def Generate_TL(self, start_pad, finish_pad, d, d1, d2, layer):
        TL = Transmission_Line(self.Pads, start_pad, finish_pad, d, d1, d2, layer)
        self.tl = TL
        self.tl_cell1 = gdspy.Cell('TL1')
        self.tl_cell2 = gdspy.Cell('TL2')
        self.tl_cell3 = gdspy.Cell('TL3')
        self.tl_cell1.add(TL.GenerateTL(d, coords = [], rots = [])) 
        self.tl_cell2.add(TL.GenerateTL(d1, coords = [], rots = [])) 
        self.tl_cell3.add(TL.GenerateTL(d2, coords = [], rots = [])) 
        #self.cell.add([gdspy.CellReference(self.tl_cell, (0, 0))])
      
    def Generate_Resonators(self, start_points, freqs, modes, ws, ss, TL_inner, TL_vacuum, TL_ground, DEs):
        self.RESONATORS1 = gdspy.Cell('RESONATORS1')
        self.RESONATORS2 = gdspy.Cell('RESONATORS2')
        self.RESONATORS3 = gdspy.Cell('RESONATORS3')
        self.RESONATORS_RESTRICTED = gdspy.Cell('RESONATORS_RESTRICTED')
        #self.resonators = []
        #self.res_cells = []
        self.res_references = []
        for i, (start, freq, mode, w, s, TLi, TLv, TLg, DE) in enumerate(zip(start_points, freqs, modes, ws, ss, TL_inner, TL_vacuum, TL_ground, DEs)):
            #self.resonators.append(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE))
            #self.res_cells.append(gdspy.Cell('res' + str(i)))
            self.RESONATORS1.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate1(mode))
            self.RESONATORS2.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate2(mode))
            self.RESONATORS3.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate3(mode))
            self.RESONATORS_RESTRICTED.add(Resonator(start.x, start.y, freq, w, s, TLi, TLv, TLg, DE).Generate3(mode))
            #self.res_cells[i].add(self.resonators[i].Generate3(mode))
            
        COMBINATION = gdspy.Cell('COMBINATION')
        COMBINATION.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS3, (0,0)), gdspy.CellReference(self.tl_cell3, (0,0)), 'or'))
        CUT = gdspy.Cell('CUT')
        CUT.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS2, (0,0)), gdspy.CellReference(self.tl_cell2, (0,0)), 'or'))
        CUTOUT = gdspy.Cell('CUTOUT')
        CUTOUT.add(gdspy.boolean(gdspy.CellReference(COMBINATION, (0,0)), gdspy.CellReference(CUT, (0,0)), 'not'))
        FILL = gdspy.Cell('FILL')
        FILL.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS1, (0,0)), gdspy.CellReference(self.tl_cell1, (0,0)), 'or'))
        
        CUTOUTFILL = gdspy.Cell('CUTOUTFILL')
        CUTOUTFILL.add(gdspy.boolean(gdspy.CellReference(CUTOUT, (0,0)), gdspy.CellReference(FILL, (0,0)), 'or'))
        self.cell.add(gdspy.CellReference(CUTOUTFILL, (0, 0)))

        
                        
    def Generate_Grid(self, a, b, l1, l2):
        self.grid_cell = gdspy.Cell('Grid')
        res1 = gdspy.Rectangle((400, 400), (404, b - 400), layer = l1) 
        x = 447
        while x < a - 400:
            r1 = gdspy.Rectangle((x, 400), (x + 4, b - 400), layer = l1) 
            x += 47
            res1 = gdspy.boolean(res1, r1, 'or')
            #self.grid_cell.add(r1)
        y = 400
        res2 = gdspy.Rectangle((400, 400), (400, 404), layer = l2) 
        while y < b - 400:
            r1 = gdspy.Rectangle((400, y), (a - 400, y + 4), layer = l2) 
            y += 47
            res2 = gdspy.boolean(res2, r1, 'or')
            #self.grid_cell.add(r1)
        res2 = gdspy.boolean(res2, res1, 'not')
        for area in self.Pads.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for area in self.tl.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        area = gdspy.CellReference(self.RESONATORS_RESTRICTED, (0,0))
        res1 = gdspy.boolean(res1, area, 'not', layer = l1)
        res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        
        #res = gdspy.boolean(res, self.Pads_cell.get_polygonsets(), 'not')
        self.grid_h = res1
        self.grid_v = res2
        self.grid_cell.add(res1)
        self.grid_cell.add(res2)
        self.cell.add(gdspy.CellReference(self.grid_cell, (0, 0)))
    #self.Resonators
    #self.JJ
    #self.Loops
    #self.TL

In [4]:
TL_inner = 5.4
TL_vacuum = 3
TL_ground = 20
TL_total = TL_inner + 2*(TL_vacuum + TL_ground)
number = 78
w = 40
s = 10
DE = 15
positions = [] 
for j in range(3):
    for i in range(13):
        if j == 0:
            positions.append(coordinates(780 + 50 + 660*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(780 + 50 + 660*i, 3900 + TL_total/2 - j*1400))
        elif j == 1:
            positions.append(coordinates(680 + 670*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(680 + 670*i, 3900 + TL_total/2 - j*1400))
        elif j == 2:
            positions.append(coordinates(570 + 660*i, 3900 + TL_total/2 - j*1400)) 
            positions.append(coordinates(570 + 660*i, 3900 + TL_total/2 - j*1400)) 
w_arr = np.full(number, w) 
s_arr = np.full(number, s) 
f_arr = np.linspace(8e9, 9e9, number)
rd.shuffle(f_arr)
print(f_arr)
DE_arr = np.full(number, DE)
TL_inner_arr = np.repeat(TL_inner, number)
TL_vacuum_arr = np.repeat(TL_vacuum, number)
TL_ground_arr = np.repeat(TL_ground, number)
ground_arr = np.repeat('ground', number)
updown = ['up', 'down']
updown_arr = np.tile(updown, number)

[8.50649351e+09 8.67532468e+09 8.49350649e+09 8.72727273e+09
 8.74025974e+09 8.66233766e+09 8.14285714e+09 8.76623377e+09
 8.31168831e+09 8.38961039e+09 8.84415584e+09 8.15584416e+09
 8.87012987e+09 8.83116883e+09 8.25974026e+09 8.59740260e+09
 8.48051948e+09 8.02597403e+09 8.96103896e+09 8.53246753e+09
 8.03896104e+09 8.55844156e+09 8.70129870e+09 8.40259740e+09
 8.85714286e+09 8.18181818e+09 8.46753247e+09 8.10389610e+09
 8.81818182e+09 8.07792208e+09 8.58441558e+09 8.57142857e+09
 8.05194805e+09 8.23376623e+09 8.01298701e+09 8.32467532e+09
 8.16883117e+09 8.93506494e+09 8.77922078e+09 8.80519481e+09
 8.68831169e+09 8.92207792e+09 8.09090909e+09 8.36363636e+09
 8.64935065e+09 8.44155844e+09 8.06493506e+09 8.62337662e+09
 8.27272727e+09 8.71428571e+09 8.00000000e+09 8.63636364e+09
 8.90909091e+09 8.79220779e+09 8.20779221e+09 8.28571429e+09
 8.29870130e+09 8.51948052e+09 8.11688312e+09 8.41558442e+09
 8.94805195e+09 8.45454545e+09 8.19480519e+09 8.75324675e+09
 8.88311688e+09 8.610389

In [5]:
sample = Sample(2)

sample.Generate_Pads(10000, 5000, #width, depth
                     2, #number of Pads
                     [TL_inner, TL_inner], #d
                     [TL_inner +2*TL_vacuum, TL_inner +2*TL_vacuum], #d1
                     [TL_total, TL_total], #d2
                     0, #layer
                     [coordinates(800, 3900-TL_total/2), coordinates(9200, 1100-TL_total/2)]) #reference coordinates for Pads
sample.Generate_TL(0, 1, TL_inner, TL_inner +2*TL_vacuum, TL_inner + 2*(TL_vacuum+TL_ground), 0)

sample.Generate_Resonators(positions, # ref coordinates
                           f_arr, #freqs
                           updown_arr, #mode
                           w_arr, #w
                           s_arr, #s
                           TL_inner_arr,
                           TL_vacuum_arr,
                           TL_ground_arr,
                           DE_arr) 


coupling length: 498.50315861651535
x coordinate of end of resonator 1470.5031586165153
DE= 15
...
coupling length: 498.50315861651535
x coordinate of end of resonator 1470.5031586165153
DE= 15
...
coupling length: 498.50315861651535
x coordinate of end of resonator 1470.5031586165153
DE= 15
...
coupling length: 498.50315861651535
x coordinate of end of resonator 1470.5031586165153
DE= 15
...
coupling length: 488.8017498410442
x coordinate of end of resonator 1460.8017498410443
DE= 15
...
coupling length: 488.8017498410442
x coordinate of end of resonator 1460.8017498410443
DE= 15
...
coupling length: 488.8017498410442
x coordinate of end of resonator 1460.8017498410443
DE= 15
...
coupling length: 488.8017498410442
x coordinate of end of resonator 1460.8017498410443
DE= 15
...
coupling length: 499.2653958621063
x coordinate of end of resonator 2131.2653958621063
DE= 15
...
coupling length: 499.2653958621063
x coordinate of end of resonator 2131.2653958621063
DE= 15
...
coupling length:

coupling length: 495.47734278272776
x coordinate of end of resonator 8067.477342782728
DE= 15
...
coupling length: 495.47734278272776
x coordinate of end of resonator 8067.477342782728
DE= 15
...
coupling length: 495.47734278272776
x coordinate of end of resonator 8067.477342782728
DE= 15
...
coupling length: 487.34264014002616
x coordinate of end of resonator 8719.342640140027
DE= 15
...
coupling length: 487.34264014002616
x coordinate of end of resonator 8719.342640140027
DE= 15
...
coupling length: 487.34264014002616
x coordinate of end of resonator 8719.342640140027
DE= 15
...
coupling length: 487.34264014002616
x coordinate of end of resonator 8719.342640140027
DE= 15
...
coupling length: 504.667030747786
x coordinate of end of resonator 8736.667030747785
DE= 15
...
coupling length: 504.667030747786
x coordinate of end of resonator 8736.667030747785
DE= 15
...
coupling length: 504.667030747786
x coordinate of end of resonator 8736.667030747785
DE= 15
...
coupling length: 504.66703

coupling length: 524.1084572934471
x coordinate of end of resonator 6706.108457293447
DE= 15
...
coupling length: 524.1084572934471
x coordinate of end of resonator 6706.108457293447
DE= 15
...
coupling length: 507.0179641208347
x coordinate of end of resonator 6689.017964120834
DE= 15
...
coupling length: 507.0179641208347
x coordinate of end of resonator 6689.017964120834
DE= 15
...
coupling length: 507.0179641208347
x coordinate of end of resonator 6689.017964120834
DE= 15
...
coupling length: 507.0179641208347
x coordinate of end of resonator 6689.017964120834
DE= 15
...
coupling length: 490.26962296369004
x coordinate of end of resonator 7342.26962296369
DE= 15
...
coupling length: 490.26962296369004
x coordinate of end of resonator 7342.26962296369
DE= 15
...
coupling length: 490.26962296369004
x coordinate of end of resonator 7342.26962296369
DE= 15
...
coupling length: 490.26962296369004
x coordinate of end of resonator 7342.26962296369
DE= 15
...
coupling length: 502.337798298

coupling length: 477.36779078043503
x coordinate of end of resonator 5149.367790780435
DE= 15
...
coupling length: 477.36779078043503
x coordinate of end of resonator 5149.367790780435
DE= 15
...
coupling length: 492.4880375472362
x coordinate of end of resonator 5164.488037547237
DE= 15
...
coupling length: 492.4880375472362
x coordinate of end of resonator 5164.488037547237
DE= 15
...
coupling length: 492.4880375472362
x coordinate of end of resonator 5164.488037547237
DE= 15
...
coupling length: 492.4880375472362
x coordinate of end of resonator 5164.488037547237
DE= 15
...
coupling length: 503.1118164773769
x coordinate of end of resonator 5835.111816477377
DE= 15
...
coupling length: 503.1118164773769
x coordinate of end of resonator 5835.111816477377
DE= 15
...
coupling length: 503.1118164773769
x coordinate of end of resonator 5835.111816477377
DE= 15
...
coupling length: 503.1118164773769
x coordinate of end of resonator 5835.111816477377
DE= 15
...
coupling length: 506.2318897

In [6]:
#gdspy.LayoutViewer(depth = 2)

In [7]:
sample.Generate_Grid(10000,5000,0,0)

In [8]:
gdspy.write_gds('resonators3_3.gds', cells=None, name='library', unit=1e-06, precision=1e-09, timestamp=None, binary_cells=None)